**Contexto inicial**

**Objetivo del analisis**

Construir un análisis exploratorio de datos que caracterice la base de datos financiera proveniente de los bancos centrales de la Unión Europea para reconocer inicialmente incidencias repetitivas asociadas a casos de fraude.


In [1]:


print("Hello queen")
    

Hello queen
